In [2]:
from langchain_groq import ChatGroq

In [74]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key= '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',
    model_name="llama-3.1-70b-versatile"
)

In [75]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobdetails.nestle.com/job/St_-Louis-Senior-Data-Scientist-MO-63164/1139607501/?feedId=256801&utm_source=NestleCareers")

page_data = loader.load().pop().page_content

In [76]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys:'role,'experience','skills' and 'description' and return only one job description.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "Senior Data Scientist",
  "experience": "Bachelor’s degree with 5 years of experience in data science, machine learning, artificial intelligence capabilities in the design, development and deployment of business solutions OR equivalent advanced degree (Master’s, Doctorate) with 2+ years of experience",
  "skills": "Python, R, Databricks, analytical, statistical, unsupervised/supervised learning, optimization, and visualization techniques",
  "description": "Apply advanced data science, machine learning, and artificial intelligence capabilities to develop and industrialize analytics applications in support of continuous improvement, management decision-making, and operational effectiveness."
}
```


In [77]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Data Scientist',
 'experience': 'Bachelor’s degree with 5 years of experience in data science, machine learning, artificial intelligence capabilities in the design, development and deployment of business solutions OR equivalent advanced degree (Master’s, Doctorate) with 2+ years of experience',
 'skills': 'Python, R, Databricks, analytical, statistical, unsupervised/supervised learning, optimization, and visualization techniques',
 'description': 'Apply advanced data science, machine learning, and artificial intelligence capabilities to develop and industrialize analytics applications in support of continuous improvement, management decision-making, and operational effectiveness.'}

In [78]:
import pandas as pd
data = pd.read_csv("portfolios.csv")
data.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [79]:
import uuid
import chromadb
chroma_client = chromadb.PersistentClient('vectorstore')

collection = chroma_client.get_or_create_collection(name="my_portfolio")

if not collection.count():
    for _, row in data.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas = {"links": row["Links"]},
                       ids = [str(uuid.uuid4())]
                      )


In [80]:
links = collection.query(query_texts=json_res['skills'],
                        n_results =2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [81]:
 prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Bharani S, a business development executive at TCS. TCS is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of TCS 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase TCS's portfolio: {link_list}
        Remember you are Bharani, BDE at TCS. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_res), "link_list": links})
print(res.content)

Subject: Unlock the Power of Data Science with TCS

Dear Hiring Manager,

I came across the Senior Data Scientist role at your esteemed organization and was impressed by the emphasis on leveraging advanced data science, machine learning, and artificial intelligence capabilities to drive business growth. As a Business Development Executive at TCS, I'd like to introduce you to our expertise in empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

At TCS, we have a proven track record of delivering cutting-edge data science solutions that cater to the evolving needs of businesses. Our team of experts is well-versed in Python, R, Databricks, and various analytical, statistical, unsupervised/supervised learning, optimization, and visualization techniques. We've successfully developed and industrialized analytics applications that support continuous improvement, management decision-making, and operati